# HW2 - Basic Data Processing for Data Mining

#### Data Science in Korea University, Prof. Jaewoo Kang
#### COSE471, Spring 2018
#### Due : 5/24 (Thur)  11:59 AM 

------------------------------------------
## __ Ready for Assingment __
-  install [python](https://www.python.org/)

- install python
     - install [anaconda](https://www.continuum.io/downloads) for science, math, and data analysis.
     
       The installation process please refer to the __blackboard__.


- install libraries
     - install libraries
         - [NumPy](http://docs.scipy.org/doc/numpy-dev/user/index.html), fot arrays
         - [pandas](http://pandas.pydata.org/), for data frames
         - [matplotlib](http://matplotlib.org/), for plotting
         - [requests](http://docs.python-requests.org/en/latest/), for fetching web content
         - [pattern](http://www.clips.ua.ac.be/pages/pattern) or [beautiful soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/), for parsing html and xml pages
         
------------------------------------------

### CHECK YOUR PYTHON VERSION
- #### Assingment made by python 3.x
- #### It will be easy for you to use same version with TA

In [1]:
import sys
sys.version

'3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]'

## Python Assingment HW2 Guide
- You need to fill out your code in ###YOUR CODE HERE### or #TODO.
- The code results should be the same as the results displayed below the code or the format we gave in problem description.
- The necessary libraries are already imported below.
    - If you don't have some libraries, install it with pip or conda command. [Google will help you]

In [ ]:
%matplotlib inline 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Problem1. Web Crawling & Processing

### Problem 1-1 Fetching the Wikipedia site

#### print html code of presented page
#### Use requests library
> *`<!DOCTYPE html>`*<br/>
> *`<html class="client-nojs" lang="en" dir="ltr">`*<br/>
> *`<head>`*<br/>
> *`<meta charset="UTF-8"/>`*<br/>
> *`<title>List of 2016 Summer Olympics medal winners - Wikipedia</title>`*<br/>
> *`...`*<br/>
> *`</script>`*<br/>
> *`</body>`*<br/>
> *`</html>`*<br/>

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_2016_Summer_Olympics_medal_winners"

###YOUR CODE HERE###
data = requests.get(url)
print(data.text)

### Problem 1-2 Parsing html data
#### Data that we are going to use is table titled below:
#### Title: Swimming men's events
#### Use BeautifulSoup libary

*print html code of parsed table: Swimming men's events*

> *`<table border="" class="class1 class2" style="font-size: 100%">`*<br/>
> *`<tr style="text-align: center;">`*<br/>
> *`<th scope="col" style="width: 120px;">Event</th>`*<br/>
> *`...`*<br/>
> *`</td>`*<br/>
> *`</tr>`*<br/>
> *`</table>`*<br/>

__*hint: class/id can be assigned for elements of html.*__

In [ ]:
#step1: look at html source!"
#step2: get all tables"
#step3: get table we care about"

###YOUR CODE HERE###
soup = BeautifulSoup(data.text, "lxml")
tables = soup.find_all('table', {'class':'wikitable plainrowheaders'})
print(tables[36])

### Problem 1-3 Extracting data and filling it into a dictionary(Python data stucture)

#### Extract  countries and medals from "Athletics - Women" table and store it in dictionary.
*Please print parsed dictionary at the end of code*

#### Shape of Dictionary
__Include only once if athlete have received the same medal more than once. ex) 100m gold Bolt, 200m gold Bolt -> gold: Bolt__
- { *country_name*: { *medal type*: [*athlete name,* *athlete name,* *athlete name,* ...], ... }}
- e.g. {'Jamaica': {'Gold': ['Elaine Thompson', 'Shaunae Miller'...], 'Silver': [...], 'Bronze': [...] }, 'United States': ... }

*__hint: Use above problem's code to get Atheletics - Women table__*

In [ ]:
# extract headings and actual data and combine data into single dict

###YOUR CODE HERE###

In [ ]:
## Get Atheletics Woman Datas
atheletics_women = tables[2]
print(atheletics_women)

In [ ]:
## Get Medal Infos
medals = atheletics_women.find_all("th")[1:]
medals = [medal.text for medal in medals]
medals

In [ ]:
## Get Medalist Infos
medalists = atheletics_women.find_all("td")
medalists = [medalist.find_all('a') for medalist in medalists if (len(medalist.find_all('a')) != 0)]

for i in range(len(medalists)):
    for j in range(len(medalists[i])):
        medalists[i][j] = medalists[i][j].text
        
        if len(medalists[i][j]) == 2:
            del medalists[i][j]

medalists = [medalists[i] for i in range(len(medalists)) if len(medalists[i]) != 0]
medalists

In [ ]:
## Split Medalists using medal type
gold_medalists = [medalists[i] for i in range(1, len(medalists), 4)]
silver_medalists = [medalists[i] for i in range(2, len(medalists), 4)]
bronze_medalists = [medalists[i] for i in range(3, len(medalists), 4)]
medalists = [gold_medalists, silver_medalists, bronze_medalists]

In [ ]:
atheletics_women_medalists = {}

In [ ]:
## Using Medal and Medalists Infos, Fill dict
for i in range(3):
    for medalist in medalists[i]:
        if len(medalist) == 2:
            if medalist[1] in atheletics_women_medalists and medals[i] in atheletics_women_medalists[medalist[1]]:
                atheletics_women_medalists[medalist[1]][medals[i]].append(medalist[0])
            else:
                if medalist[1] in atheletics_women_medalists:
                    atheletics_women_medalists[medalist[1]][medals[i]] = [medalist[0]]
                else:
                    atheletics_women_medalists[medalist[1]] = {medals[i]: [medalist[0]]}
        else:
            if medalist[0] in atheletics_women_medalists and medals[i] in atheletics_women_medalists[medalist[0]]:
                atheletics_women_medalists[medalist[0]][medals[i]].extend(medalist[1:])
            else:
                if medalist[0] in atheletics_women_medalists:
                    atheletics_women_medalists[medalist[0]][medals[i]] = medalist[1:]
                else:
                    atheletics_women_medalists[medalist[0]] = {medals[i]: medalist[1:]}

In [ ]:
## Delete Duplicate Data
for country in atheletics_women_medalists:
    for medal in medals:
        if medal in atheletics_women_medalists[country]:
            atheletics_women_medalists[country][medal] = list(set(atheletics_women_medalists[country][medal]))

In [ ]:
atheletics_women_medalists

### Problem 1-4 Creating a dataframe(Python data structure) from a dictionary

#### Use Dictionary that you made in problem 1-3

#### Use Pandas library

__* Shape of dataframe must be like following table.__
<table>
  <tr>
    <th></th>
    <th>Gold</th>
    <th>Silver</th>
    <th>Bronze</th>
  </tr>
  <tr>
    <td>Jamaica</td>
    <td>Elaine Thompson, Shaunae Miller ... </td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
  </tr>
  <tr>
    <td>United States</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
  </tr>
  <tr>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
  </tr>
</table>

In [ ]:
# create dataframe
# Use the following code
# indices: countries, columns: Medal type

###YOUR CODE HERE###
df = pd.DataFrame.from_dict(atheletics_women_medalists, orient='index')
df

### Problem 1-5 Sort Data Frame
### Sort Country names by number of Gold medal athletes (Descending order)
- __Show in table form (e.g. pd.DataFrame.describe())__
<table>
  <tr>
    <th></th>
    <th>Gold</th>
    <th>Silver</th>
    <th>Bronze</th>
  </tr>
  <tr>
    <td>Jamaica</td>
    <td>Elaine Thompson, Shaunae Miller ... </td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
  </tr>
  <tr>
    <td>United States</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
  </tr>
  <tr>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
  </tr>
</table>

In [ ]:
###YOUR CODE HERE###

## Get Gold_count
tmp = df['Gold'].fillna('0')
Gold_count = []
for i in tmp:
    if i is not '0':
        Gold_count.append(len(i))
    else:
        Gold_count.append(0)

In [ ]:
## Insert Gold_count in df
df.insert(loc = 3, column = 'count', value = Gold_count)

In [ ]:
## Sort df using Gold_count
df.sort_values(by='count', ascending=False, inplace=True)

In [ ]:
## Delete Gold_count column that is unused
del df['count']

In [ ]:
df

### Problem 1-6 Handle Data Frame
### Which country has the most bronze medals in Atheletics - Women? 
- __Make sub data frame__
- __Show in table form (e.g. pd.DataFrame.describe())__
<table>
  <tr>
    <th></th>
    <th>Bronze</th>
  </tr>
  <tr>
    <td>United States</td>
    <td>Tori Bowie ...</td>
</table>

In [ ]:
###YOUR CODE HERE###

## Get bronze_count
tmp = df['Bronze'].fillna('0')
Bronze_count = []
for i in tmp:
    if i is not '0':
        Bronze_count.append(len(i))
    else:
        Bronze_count.append(0)

In [ ]:
## Insert Bronze_count in df
df.insert(loc = 3, column = 'Bronze_count', value = Bronze_count)

In [ ]:
## Sort df using Bronze_count
df.sort_values(by='Bronze_count', ascending=False, inplace=True)

In [ ]:
pd.DataFrame(df['Bronze'])[:1]

## Project Problem 1
- __URL : https://en.wikipedia.org/wiki/List_of_Intel_microprocessors __
- __Crawl "7th generation Core/Kaby Lake/Skylake (X-series Processors)/Apollo Lake - desktop" table in given URL__
- __Make data frame same as in wiki page with cralwed table (Show in table form)__
- __Sort model name by "Base Frequency" (Show in table form)__ 

In [ ]:
###YOUR CODE HERE###

## Get Wiki Data
url = "https://en.wikipedia.org/wiki/List_of_Intel_microprocessors"

data = requests.get(url)
soup = BeautifulSoup(data.text, "lxml")

In [ ]:
###YOUR CODE HERE###

## Get Table
tables = soup.find_all('table', {'class':'wikitable sortable'})
table = tables[2]
table

In [ ]:
## Get Table Attributes
attributes = table.find_all("tr")[0].find_all("th")
attributes = [attribute.text for attribute in attributes]
attributes

In [ ]:
## Get Table Records
records = table.find_all("tr")[1:]
records = [record.find_all("td") for record in records]
records = [[record.text for record in records[i]] for i in range(len(records))]
records

In [ ]:
###YOUR CODE HERE###

## Make DataFrame
df = pd.DataFrame(records, columns = attributes)
df

In [ ]:
###YOUR CODE HERE###

## Sort by base frequency
df.sort_values(by='Base frequency (GHz)', ascending=True, inplace=True)
df

In [ ]:
###YOUR CODE HERE###

## Show model name sorted by Base Frequency
pd.DataFrame(df['Model'])

## Problem2. Processing Data with Data Frame

### Fetching data from tsv(tab-separated values)
We will give you GDP per capita in tsv format.
(Gross domestic product per capita, current prices in USD; International Monetary Fund, World Economic Outlook Database, October 2016) 

What you have to do is read data from tsv, put it in pandas dataframe, manipulate data so that it is ready for analyse.

In [ ]:
tsv = pd.read_csv('GDPperCapByTab.txt',sep='\t',index_col=0) 
# this will let you read data from file name ~~ , and understand <tab> as delimiter

tsv # You can check whether data is correctly loaded in your df(;tsv) or not

### Problem 2-1 Data Cleaning
1. Drop everything but "name of country" and "data by year(1980 ~ 2010)".
2. Modify data so that you can change it to float type number. (For example, u'1,421.84' can not be changed to float while "," exists.) 'NaN' can be changed to np.nan.
<br><b>Hint, you can change type of whole data to float in one instuction if your data is in good format. </b>

In [ ]:
###YOUR CODE HERE###

#TODO: Drop everything but "name of country" and "data by year(1980 ~ 2010)".
tsv.drop(columns = ['Subject Descriptor', 'Units', 'Scale', 'Country/Series-specific Notes', '2011', '2012', '2013', '2014', '2015', 'Estimates Start After'], inplace = True)

#TODO: modify data suitable to change data type, like n/a to np.nan, etcs
tsv.fillna(np.nan, inplace = True)

In [ ]:
drop_index = []

#TODO: modify data type as float type
for columnNum in [str(num) for num in range(1980, 2011)]:
    for i in range(len(tsv[columnNum])):
        if isinstance(tsv[columnNum][i], str):
            tsv[columnNum][i] = np.float(tsv[columnNum][i].replace(',', ''))
        else:
            drop_index.append(tsv.index[i])

## Delete Nan rows
drop_index = list(set(drop_index))
tsv.drop(drop_index, inplace = True)

In [ ]:
tsv

In [ ]:
print(tsv.index[0],tsv.iloc[0,0], type(tsv.ix[0,0]))

### Problem 2-2 Data  Grouping
### Get median value by country of each 5 years from 1980 to 2010
- __Condition 1 : Data should be in "int" type (numpy.int64 or else)__ 
- __Condition 2 : Dataframe should be in the form of given output (two cells below)__

In [ ]:
resulttsv = tsv
# Think wisely! It does not require long code. For me, 3 lines were enough. You can do it :)

###YOUR CODE HERE###

## Data -> Int type
resulttsv = resulttsv.astype('int64')

## ColumnNums
columnNums = [
['1980', '1981', '1982', '1983', '1984', '1985'],
['1985', '1986', '1987', '1988', '1989', '1990'],
['1990', '1991', '1992', '1993', '1994', '1995'],
['1995', '1996', '1997', '1998', '1999', '2000'],
['2000', '2001', '2002', '2003', '2004', '2005'],
['2005', '2006', '2007', '2008', '2009', '2010']
]

## Get Median value by country of each 5 years
for columnNum in columnNums:
    resulttsv[str(columnNum[0]) + '-' + str(columnNum[-1])] = resulttsv[columnNum].median(axis = 'columns')
    resulttsv.drop(columnNum[:5], 1, inplace = True)
resulttsv.drop('2010', 1, inplace = True)

## Data -> Int type
resulttsv = resulttsv.astype('int64')

In [ ]:
print(resulttsv.index[1],resulttsv.ix[1,0],type(resulttsv.ix[1,0]))
resulttsv.head()

### Problem 2-3 Data  Filtering
### Filter Rows Where Median Value(1995 - 2000) Is Greater Than 10000
- __Hint, If you are not familiar with data filtering using pandas.dataframe, Please refer to the following link below  . __
    - __https://pythonspot.com/pandas-filter/__

In [ ]:
###YOUR CODE HERE###
filtered_tsv = resulttsv[(resulttsv['1995-2000'] > 10000)]

In [ ]:
print(filtered_tsv.index[1],filtered_tsv.ix[1,0],type(filtered_tsv.ix[1,0]))
filtered_tsv.head()

### Problem 2-4 Get Infant Mortality rate

### We need data from 1980-2010. Drop data which is not necessary


In [ ]:
IMtsv = pd.read_csv('InfantMortalityByTab.txt',sep='\t',index_col=0)
###YOUR CODE HERE###

In [ ]:
IMtsv.index

In [ ]:
columnNames = [str(x) + '-' + str(x+5) for x in range(1980, 2010, 5)]
DroppedIMtsv = IMtsv[columnNames]

In [ ]:
DroppedIMtsv.head()

## Problem3. Draw data as a plot

### Problem 3-1 Define a function to draw catter plot which contained trendline.
#### use .scatter from matplotlib library.
- please print *<b>line equation</b>* and *<b>MSE(mean squared error)</b>* for each plots
- matplotlib.pyplot.scatter: http://matplotlib.org/api/pyplot_api.html?highlight=scatter#matplotlib.pyplot.scatter
- you can find example of trendline on matplotlib very easily.

In [ ]:
def plot_2D_trendline(x_data, y_data, pred=None, x_label='X', y_label='Y', title=None):
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.scatter(x_data, y_data)
    
    if pred is not None:
        plt.plot(x_data, pred, color='red', linewidth=2)
    if title is not None:
        plt.title(title)
    
    # the line equation:
    a, b = np.polyfit(x_data, y_data, 1)
    p = np.poly1d([a, b])
    plt.plot(x_data, p(x_data),"r--")
    print ("y=%.6fx+(%.6f)" % (a, b))
    
    #print Mean Squared Error
    mse = np.square(np.subtract(y_data, p(x_data))).mean()
    print ("MeanSquaredError: ", mse)
    
    plt.show()

# Let's see if the function works.
plot_2D_trendline(np.random.randint(1000,size=100),np.random.randint(1000,size=100))

### Problem 3-2 Draw scatter plot and trendline by using infant mortality rate and GDP per capita
- x-axis: *GDP per cap*, y-axis: *infant mortality*
- one plot per 5-year period (total 7 plots)
- The trend line may seem somewhat meaningless, but it isn't problem now.

In [ ]:
for country in DroppedIMtsv.index:
    if country not in resulttsv.index:
        DroppedIMtsv.drop(country, inplace = True)

for country in resulttsv.index:
    if country not in DroppedIMtsv.index:
        resulttsv.drop(country, inplace = True)

In [ ]:
DroppedIMtsv.head()

In [ ]:
resulttsv.head()

In [ ]:
for years in DroppedIMtsv.columns:
    xData=np.array(resulttsv[years])
    yData=np.array(DroppedIMtsv[years])
    
    #TODO
    plot_2D_trendline(xData, yData, x_label='GDP per Cap', y_label='Infant Mortality', title=years)

### Problem 3-3 Let's make trend line meaningful.
- As mentioned, the trend line is somewhat meaningless.
- By applying a *<b>logarithmic scale</b>* on the data, you can make trendlines more meaningful.
<br/>    ** hint: Most of mathmetical functions in python can be found in numpy.*

In [ ]:
for years in DroppedIMtsv.columns:
    xData=np.array(resulttsv[years])
    yData=np.array(DroppedIMtsv[years])
    
    #TODO
    xDataLog = np.log(xData)
    yDataLog = np.log(yData)
    plot_2D_trendline(xDataLog, yDataLog, x_label='GDP per Cap(Log Scale)', y_label='Infant Mortality (Log Scale)', title=years+" (Log Scale)")

## Project Problem 2&3
- __Read Data forestfires.csv. Keep it mind this data is not tsv!__
- __Sorting & Filtering Data, each sub questions are independent, not related to each other. __
    - Filter rows which event occured in 1st quarter (January to April).
    - Filter rows where area is greater than average area value, and report average value also.
    - Sort descending order by FFMC value which event occured in Friday.
- __Draw plot with following directions, each sub questions are independent, not related to each other.__
    - Draw plot only using X, Y value, so we can know which part of park frequently fired.
    - Draw plot only using temp and wind, so we can know the relation between two features. Also draw trend line.
    - Draw plot only using temp and RH, so we can know the relation between two features. Also draw trend line.
    - Draw plot only using wind and RH, so we can know the relation between two features. Also draw trend line.

In [ ]:
###YOUR CODE HERE###

In [ ]:
FF = pd.read_csv('forestfires.csv')

In [ ]:
FF.head()

In [ ]:
filtered_month = ['jan', 'feb', 'mar', 'apr']

In [ ]:
for month in filtered_month:
    FF = FF[FF.month != month]

In [ ]:
###YOUR CODE HERE###

In [ ]:
FF = pd.read_csv('forestfires.csv')

In [ ]:
FF_area_mean = FF['area'].mean()
FF_area_mean

In [ ]:
FF = FF[FF.area > FF_area_mean]

In [ ]:
FF.head()

In [ ]:
###YOUR CODE HERE###

In [ ]:
FF = pd.read_csv('forestfires.csv')

In [ ]:
FF = FF[FF.day == 'fri']

In [ ]:
FF.sort_values(by = 'FFMC', ascending = False, inplace = True)

In [ ]:
FF.head()

In [ ]:
###YOUR CODE HERE###

In [ ]:
FF = pd.read_csv('forestfires.csv')

In [ ]:
plt.scatter(FF['X'], FF['Y'])

In [ ]:
###YOUR CODE HERE###
plot_2D_trendline(FF['temp'], FF['wind'], x_label='temp', y_label='wind')

In [ ]:
###YOUR CODE HERE###
plot_2D_trendline(FF['temp'], FF['RH'], x_label='temp', y_label='RH')

In [ ]:
###YOUR CODE HERE###
plot_2D_trendline(FF['wind'], FF['RH'], x_label='wind', y_label='RH')

# FINISH